In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp chunkadelic

# chunkadelic


> Console script and callable function for preprocessing dataset of disparate-sized audio files into uniform chunks

Note: Duplicates the directory structure(s) referenced by input paths. 

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import argparse 
import os 
from functools import partial
from tqdm.contrib.concurrent import process_map  
import torch
import torchaudio
import math
from aeiou.core import is_silence, load_audio, makedir, get_audio_filenames, normalize_audio, get_dbmax
from aeiou.chunk_one_file import chunk_one_file
import multiprocessing as mp
from multiprocessing import Pool, cpu_count, Barrier

In [ ]:
#|export
def blow_chunks(
    audio:torch.tensor,  # long audio file to be chunked
    new_filename:str,    # stem of new filename(s) to be output as chunks
    chunk_size:int,      # how big each audio chunk is, in samples
    sr=48000,            # audio sample rate in Hz
    norm='False',      # normalize input audio, based on the max of the absolute value ['global','channel', or anything else for None, e.g. False]
    spacing=0.5,         # fraction of each chunk to advance between hops
    strip=False,    # strip silence: chunks with max power in dB below this value will not be saved to files
    thresh=-70,      # threshold in dB for determining what counts as silence
    debug=False,     # print debugging information 
    ):
    "chunks up the audio and saves them with --{i} on the end of each chunk filename"
    if (debug): print(f"       blow_chunks: audio.shape = {audio.shape}",flush=True)
        
    chunk = torch.zeros(audio.shape[0], chunk_size)
    _, ext = os.path.splitext(new_filename)
    
    if norm in ['global','channel']:  audio = normalize_audio(audio, norm)     

    spacing = 0.5 if spacing == 0 else spacing # handle degenerate case as a request for the defaults
    
    start, i = 0, 0
    while start < audio.shape[-1]:
        out_filename = new_filename.replace(ext, f'--{i}'+ext) 
        end = min(start + chunk_size, audio.shape[-1])
        if end-start < chunk_size:  # needs zero padding on end
            chunk = torch.zeros(audio.shape[0], chunk_size)
        chunk[:,0:end-start] = audio[:,start:end]
        if (not strip) or (not is_silence(chunk, thresh=thresh)):
            torchaudio.save(out_filename, chunk, sr)
        else:
            print(f"Skipping chunk {out_filename} because it's 'silent' (below threhold of {thresh} dB).",flush=True)
        start, i = start + int(spacing * chunk_size), i + 1
    return 

In [ ]:
#|export   
def chunk_one_file(
    filenames:list,      # list of filenames from which we'll pick one
    args,                # output of argparse
    file_ind             # index from filenames list to read from
    ):
    "this chunks up one file by setting things up and then calling blow_chunks"
    filename = filenames[file_ind]  # this is actually input_path+/+filename
    output_path, input_paths = args.output_path, args.input_paths
    new_filename = None
    if args.debug: print(f" --- process_one_file: filenames[{file_ind}] = {filename}\n", flush=True)
    
    for ipath in input_paths: # set up the output filename & any folders it needs
        if args.nomix and ('Mix' in ipath) and ('Audio Files' in ipath): return  # this is specific to the BDCT dataset, otherwise ignore
        if ipath in filename:
            last_ipath = ipath.split('/')[-1]           # get the last part of ipath
            clean_filename = filename.replace(ipath,'') # remove all of ipath from the front of filename
            new_filename = f"{output_path}/{last_ipath}/{clean_filename}".replace('//','/') 
            makedir(os.path.dirname(new_filename))      # we might need to make a directory for the output file
            break

    if new_filename is None:
        print(f"ERROR: Something went wrong with name of input file {filename}. Skipping.",flush=True) 
        return 
    
    try:
        if args.debug: print(f"   About to load filenames[{file_ind}] = {filename}\n", flush=True)
        audio = load_audio(filename, sr=args.sr, verbose=args.debug)
        if args.debug: print(f"   We loaded the audio, audio.shape = {audio.shape}\n   Calling blow_chunks...", flush=True)
        blow_chunks(audio, new_filename, args.chunk_size, sr=args.sr, spacing=args.spacing, strip=args.strip, thresh=args.thresh, debug=args.debug)
    except Exception as e: 
        print(f"Error '{e}' while loading {filename} or writing chunks. Skipping.", flush=True)

    if args.debug: print(f" --- File {file_ind}: {filename} completed.\n", flush=True)
    return

Testing equential execution of for one file at a time, sequentially:

In [ ]:
#| eval: false
class AttrDict(dict): # cf. https://stackoverflow.com/a/14620633/4259243
    "setup an object to hold args"
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
args = AttrDict()  # setup something akin to what argparse gives
args.update( {'output_path':'test_chunks', 'input_paths':['examples/'], 'sr':48000, 'chunk_size':131072, 'spacing':0.5,
    'norm':'global', 'strip':False, 'thresh':-70, 'nomix':False, 'verbose':True,
    'workers':min(32, os.cpu_count() + 4), 'debug':True })

filenames = get_audio_filenames(args.input_paths)
print("filenames =",filenames)
for i in range(len(filenames)):
    print(f"file {i+1}/{len(filenames)}: {filenames[i]}:")
    chunk_one_file(filenames, args, i)

filenames = ['examples/stereo_pewpew.mp3', 'examples/example.wav']
file 1/2: examples/stereo_pewpew.mp3:
 --- process_one_file: filenames[0] = examples/stereo_pewpew.mp3

   About to load filenames[0] = examples/stereo_pewpew.mp3

Resampling examples/stereo_pewpew.mp3 from 44100.0 Hz to 48000 Hz
   We loaded the audio, audio.shape = torch.Size([2, 236983])
   Calling blow_chunks...
       blow_chunks: audio.shape = torch.Size([2, 236983])
 --- File 0: examples/stereo_pewpew.mp3 completed.

file 2/2: examples/example.wav:
 --- process_one_file: filenames[1] = examples/example.wav

   About to load filenames[1] = examples/example.wav

Resampling examples/example.wav from 44100 Hz to 48000 Hz
   We loaded the audio, audio.shape = torch.Size([1, 55728])
   Calling blow_chunks...
       blow_chunks: audio.shape = torch.Size([1, 55728])
 --- File 1: examples/example.wav completed.



The main executable `chunkadelic` does the same as the previous sequential execution, albeit in parallel. 

> Note: Restrictions in Python's `ProcessPoolExecutor` prevent directly invoking parallel execution of `chunk_one_file` while in interactive mode or inside a Jupyter notebook: You must use the CLI (or subprocess it). 

In [ ]:
#|export
def main():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--chunk_size', type=int, default=2**17, help='Length of chunks')
    parser.add_argument('--sr', type=int, default=48000, help='Output sample rate')
    parser.add_argument('--norm', default='False', const='False', nargs='?', choices=['False', 'global', 'channel'],
                   help='Normalize audio, based on the max of the absolute value [global/channel/False]')
    parser.add_argument('--spacing', type=float, default=0.5, help='Spacing factor, advance this fraction of a chunk per copy')
    parser.add_argument('--strip', action='store_true', help='Strips silence: chunks with max dB below <thresh> are not outputted')
    parser.add_argument('--thresh', type=int, default=-70, help='threshold in dB for determining what constitutes silence')
    parser.add_argument('--workers', type=int, default=min(32, os.cpu_count() + 4), help='Maximum number of workers to use (default: all)')
    parser.add_argument('--nomix', action='store_true',  help='(BDCT Dataset specific) exclude output of "*/Audio Files/*Mix*"')
    parser.add_argument('output_path', help='Path of output for chunkified data')
    parser.add_argument('input_paths', nargs='+', help='Path(s) of a file or a folder of files. (recursive)')
    parser.add_argument('--verbose', action='store_true',  help='Extra output logging')
    parser.add_argument('--debug', action='store_true',  help='Extra EXTRA output logging')
    args = parser.parse_args()
   
    if args.verbose: 
        print("chunkadelic: args = ",args)
        print("Getting list of input filenames")
    filenames = get_audio_filenames(args.input_paths)
    if args.verbose:
        print(f"  Got {len(filenames)} input filenames") 
        if not (args.norm in ['global','channel']): 
            print(f"Warning: since norm = {args.norm}, no normalizations will be performed.")
        print("Processing files (in parallel)...")
            
    wrapper = partial(chunk_one_file, filenames, args)
    r = process_map(wrapper, range(len(filenames)), chunksize=1, max_workers=args.workers)  # different chunksize used by tqdm. max_workers is to avoid annoying other ppl
  
    if args.verbose: print("Finished")      

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

In [ ]:
#| eval: false
! chunkadelic -h

usage: chunkadelic [-h] [--chunk_size CHUNK_SIZE] [--sr SR]
                   [--norm [{False,global,channel}]] [--spacing SPACING]
                   [--strip] [--thresh THRESH] [--workers WORKERS] [--nomix]
                   [--verbose] [--debug]
                   output_path input_paths [input_paths ...]

positional arguments:
  output_path           Path of output for chunkified data
  input_paths           Path(s) of a file or a folder of files. (recursive)

optional arguments:
  -h, --help            show this help message and exit
  --chunk_size CHUNK_SIZE
                        Length of chunks (default: 131072)
  --sr SR               Output sample rate (default: 48000)
  --norm [{False,global,channel}]
                        Normalize audio, based on the max of the absolute
                        value [global/channel/False] (default: False)
  --spacing SPACING     Spacing factor, advance this fraction of a chunk per
                        copy (default: 0.5)
  --strip


---
Testing of CLI run:  (don't run this on GitHub CI or it will hang)

In [ ]:
#| eval: false
import subprocess

In [ ]:
#| eval: false
print("chunkadelic: running tests of normalizations:")
for norm in ['False', 'global','channel']:
    print("-----\n norm =",norm)
    result = subprocess.run(['chunkadelic', '--norm', norm, 'test_chunks','examples/'], stdout=subprocess.PIPE)
    out = result.stdout.decode("utf-8") 
    print(out)
    assert 'error' not in out.lower(), f'Error occured while running with norm={norm}'  # for CI testing

chunkadelic: running tests of normalizations:
-----
 norm = False


100%|██████████| 2/2 [00:00<00:00,  3.28it/s]



-----
 norm = global


100%|██████████| 2/2 [00:00<00:00,  2.94it/s]



-----
 norm = channel


100%|██████████| 2/2 [00:00<00:00,  2.96it/s]
